In [ ]:
import os
import datetime

first_day = datetime.datetime(1997,11,10,14)
day_offset = {"A": -2, "B": -1, "C": 0, "D": 1, "E": 2, "F": 3, "G": 4}

main_dir = "/Users/Julian/Documents/WorkDocuments/Z24-S101-mini-project/S101-and-Z24-demonstrator/Z24_bridge_KUL/accelerations"

sub_dirs = ["/Z24ems1", "/Z24ems2", "/Z24ems3"]

# find data directories in first sub directory
data_dir = next(os.walk(f"{main_dir}{sub_dirs[0]}"))[1]

# find data files withing the first data directory in the first sub directory
data_files = next(os.walk(f"{main_dir}{sub_dirs[0]}/{data_dir[0]}"))[2]

# print week, day, hour info for the data directory in question
week = data_dir[0][0:2]
day = data_dir[0][2]
hour = data_dir[0][3:]
print(f"week : {week}\nday : {day}\nhour : {hour}")

# calculate the day and time that the data was collected on
days_into_project = datetime.timedelta(hours = int(hour) - 14,days = day_offset[day], weeks = int(week) - 1)
new_date = first_day + days_into_project
print(new_date)

with open(f"{main_dir}{sub_dirs[0]}/{data_dir[0]}/{data_files[0]}") as file:
	print(f"{main_dir}{sub_dirs[0]}/{data_dir[0]}/{data_files[0]}")
	end_reached = False
	for i, line in enumerate(file):
		if i == 2:
			print(f"time interval: {line}")
		elif i >= 3:
			pass
		if i == 65538:
			print(f"final sample is: {line}")
			end_reached = True
		if line.startswith("Segment #1 Start"):
			print(line)
			segment_start_str = line.replace("Segment #1 Start :","").replace("\n", "")
			segment_start_date_time = datetime.datetime.strptime(segment_start_str, "%a %b %d %H:%M:%S %Y")
			print(segment_start_date_time)

for dir in sub_dirs:
	data_dirs = next(os.walk(f"{main_dir}{dir}"))[1]
	for dir in data_dirs:
		pass